In [1]:
import pandas as pd
import requests
import os
import time
import re
from requests.exceptions import RequestException

# Substitua com sua chave de API do CORE
API_KEY = "XfCz461qd3BMH8NDt0GRWLjgUycbOs5k" # Mantenha sua chave aqui

# Caminho para sua planilha CSV de entrada
PLANILHA_ENTRADA = "/work/CenaNoink/planilha30_31_32(Sheet1).csv"

# Nome da coluna que contém os DOIs
COLUNA_DOI = "DOI"

# Arquivo CSV de saída para os resultados
ARQUIVO_SAIDA_RESULTADOS = "resultados_links_download_core_v5_rate_limit.csv"

# Configurações para o retry em caso de rate limit (erro 429)
MAX_RETRIES = 5
INITIAL_BACKOFF_SECONDS = 60 # Começa esperando 1 minuto se não houver Retry-After
MAX_BACKOFF_SECONDS = 300 # Espera máxima de 5 minutos

def obter_link_download_pdf_core(entity_id, entity_type="outputs"):
    """
    Constrói e retorna o link de download direto para o PDF de uma entidade (outputs ou works) da API CORE.
    """
    if not entity_id:
        # print(f"ID da entidade inválido para gerar link de download: {entity_id}") # Log menos verboso
        return None
    url_download = f"https://api.core.ac.uk/v3/{entity_type}/{entity_id}/pdf"
    # print(f"Link de download gerado para {entity_type} ID {entity_id}: {url_download}") # Log menos verboso
    return url_download

def buscar_entidade_por_doi(doi, entity_type="outputs"):
    """
    Busca uma entidade (outputs ou works) pelo DOI, com tratamento de rate limit (429).
    Retorna o ID da entidade, mensagem de erro (se houver), e lista de outputs para works.
    """
    query = f'doi:"{doi}"'
    url = f"https://api.core.ac.uk/v3/search/{entity_type}"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    params = {"q": query, "limit": 1}
    
    current_retry = 0
    backoff_time = INITIAL_BACKOFF_SECONDS

    while current_retry < MAX_RETRIES:
        try:
            print(f"Buscando {entity_type} com DOI: {doi} (Tentativa {current_retry + 1}/{MAX_RETRIES})")
            response = requests.get(url, headers=headers, params=params, timeout=30) # Timeout aumentado
            response.raise_for_status() # Levanta erro para 4xx/5xx, exceto se tratado abaixo
            data = response.json()

            if data.get("results") and len(data["results"]) > 0:
                result = data["results"][0]
                entity_id = result.get("id")
                if not entity_id:
                    msg = f"ID não encontrado na resposta da API para {entity_type} DOI {doi}"
                    print(msg)
                    return None, msg, None
                
                print(f"{entity_type.capitalize()} ID encontrado: {entity_id} para DOI: {doi}")
                associated_outputs = result.get("outputs", []) if entity_type == "works" else None
                return str(entity_id), None, associated_outputs
            else:
                msg = f"Nenhum resultado encontrado para {entity_type} com DOI {doi}."
                print(msg)
                return None, msg, None

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429: # Too Many Requests
                print(f"Erro 429 (Too Many Requests) ao buscar {entity_type} para DOI {doi}.")
                retry_after_header = e.response.headers.get("Retry-After")
                wait_time = backoff_time

                if retry_after_header:
                    try:
                        wait_time = int(retry_after_header)
                        print(f"Cabeçalho Retry-After encontrado. Esperando por {wait_time} segundos.")
                    except ValueError:
                        print(f"Não foi possível converter Retry-After ('{retry_after_header}') para inteiro. Usando backoff padrão de {wait_time}s.")
                else:
                    print(f"Sem cabeçalho Retry-After. Usando backoff de {wait_time} segundos.")
                
                wait_time = min(wait_time, MAX_BACKOFF_SECONDS) # Garante que não espere demais
                
                if current_retry < MAX_RETRIES - 1:
                    print(f"Tentando novamente em {wait_time} segundos...")
                    time.sleep(wait_time)
                    current_retry += 1
                    # Para o próximo backoff, se não houver Retry-After, dobramos o tempo (limitado por MAX_BACKOFF_SECONDS)
                    backoff_time = min(backoff_time * 2, MAX_BACKOFF_SECONDS) 
                else:
                    msg = f"Máximo de retries ({MAX_RETRIES}) atingido para DOI {doi} após erro 429."
                    print(msg)
                    return None, msg, None
            
            elif e.response.status_code == 404: # Not Found específico da busca
                msg = f"404 Not Found (busca {entity_type}) para DOI {doi}."
                print(msg)
                return None, msg, None
            else: # Outros erros HTTP
                msg = f"Erro HTTP {e.response.status_code} ao buscar {entity_type} para DOI {doi}: {e}"
                print(msg)
                return None, msg, None
        
        except requests.exceptions.RequestException as e: # Erros de conexão, timeout, etc.
            msg = f"Erro de requisição ao buscar {entity_type} para DOI {doi} (Tentativa {current_retry + 1}): {e}"
            print(msg)
            if current_retry < MAX_RETRIES - 1:
                print(f"Tentando novamente em {backoff_time} segundos devido a erro de requisição...")
                time.sleep(backoff_time)
                current_retry += 1
                backoff_time = min(backoff_time * 2, MAX_BACKOFF_SECONDS)
            else:
                msg_final = f"Máximo de retries ({MAX_RETRIES}) atingido para DOI {doi} após erro de requisição."
                print(msg_final)
                return None, msg_final, None
        
        except Exception as e: # Outros erros (ex: JSONDecodeError)
            msg = f"Erro inesperado ao buscar {entity_type} para DOI {doi} (Tentativa {current_retry + 1}): {e}"
            print(msg)
            # Para erros de JSON, geralmente não adianta tentar novamente com a mesma requisição
            return None, msg, None
            
    return None, f"Falha ao buscar {entity_type} para DOI {doi} após {MAX_RETRIES} tentativas.", None


if __name__ == "__main__":
    try:
        df = None
        codificacoes = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        for codificacao in codificacoes:
            try:
                df = pd.read_csv(PLANILHA_ENTRADA, encoding=codificacao)
                print(f"Arquivo CSV lido com sucesso usando a codificação: {codificacao}")
                break
            except UnicodeDecodeError:
                print(f"Falha ao ler CSV com codificação: {codificacao}. Tentando próxima...")
            except FileNotFoundError:
                print(f"ERRO: Arquivo de entrada não encontrado em: {PLANILHA_ENTRADA}")
                exit()

        if df is None:
            raise Exception(f"Não foi possível decodificar o arquivo CSV: {PLANILHA_ENTRADA} com as codificações testadas: {codificacoes}")

        resultados_finais = []

        if COLUNA_DOI not in df.columns:
            print(f"ERRO: Coluna DOI '{COLUNA_DOI}' não encontrada na planilha.")
            exit()

        print(f"\nIniciando processamento de DOIs da coluna: '{COLUNA_DOI}'")
        total_dois = len(df)
        for index, row in df.iterrows():
            doi = row[COLUNA_DOI]
            link_download_encontrado = None
            status_final = "Falha"
            motivo_falha_final = "DOI não processado"
            id_usado_para_link = None
            tipo_entidade_link = None
            
            # Inicializa os campos de tentativa
            core_id_output_tentado = None
            core_id_work_tentado = None
            core_id_output_do_work_tentado = None


            print(f"\n--- Processando DOI {index + 1}/{total_dois}: {doi} (Linha {index+2} da planilha) ---")

            if pd.isna(doi) or str(doi).strip() == "":
                print(f"DOI vazio ou NaN. Pulando.")
                motivo_falha_final = "DOI vazio ou NaN"
            else:
                doi_str = str(doi).strip()

                # Tentativa 1: Buscar e obter link de "Outputs"
                output_id, erro_busca_output, _ = buscar_entidade_por_doi(doi_str, "outputs")
                core_id_output_tentado = output_id # Registra a tentativa
                if output_id:
                    link_download_encontrado = obter_link_download_pdf_core(output_id, "outputs")
                    if link_download_encontrado:
                        status_final = "Sucesso"
                        motivo_falha_final = "Link obtido de Output direto"
                        id_usado_para_link = output_id
                        tipo_entidade_link = "outputs"
                else: # Se output_id é None
                    motivo_falha_final = f"Output não encontrado ou erro na busca: {erro_busca_output}"

                # Tentativa 2: Buscar e obter link de "Works" (se a tentativa 1 falhou)
                if not link_download_encontrado:
                    print(f"Fallback para 'Works' para DOI: {doi_str}")
                    work_id, erro_busca_work, associated_output_ids = buscar_entidade_por_doi(doi_str, "works")
                    core_id_work_tentado = work_id # Registra a tentativa
                    if work_id:
                        # Tentativa 2a: Obter link direto do Work
                        print(f"Tentando obter link do Work ID: {work_id}")
                        link_download_encontrado = obter_link_download_pdf_core(work_id, "works")
                        if link_download_encontrado:
                            status_final = "Sucesso"
                            motivo_falha_final = "Link obtido de Work direto"
                            id_usado_para_link = work_id
                            tipo_entidade_link = "works"
                        else:
                            motivo_falha_final = f"Falha ao gerar link do Work ID {work_id}."
                            
                            if associated_output_ids:
                                print(f"Work ID {work_id}. Tentando obter link de {len(associated_output_ids)} outputs associados.")
                                for assoc_output_id in associated_output_ids:
                                    core_id_output_do_work_tentado = assoc_output_id
                                    print(f"Tentando Output associado ID: {assoc_output_id} do Work ID: {work_id}")
                                    link_download_encontrado = obter_link_download_pdf_core(assoc_output_id, "outputs")
                                    if link_download_encontrado:
                                        status_final = "Sucesso"
                                        motivo_falha_final = f"Link obtido de Output ID {assoc_output_id} (associado ao Work ID {work_id})"
                                        id_usado_para_link = assoc_output_id
                                        tipo_entidade_link = "outputs (de work)"
                                        break 
                                if not link_download_encontrado:
                                    motivo_falha_final = f"Falha ao obter link de Work direto e de todos os {len(associated_output_ids)} outputs associados ao Work ID {work_id}."
                            elif not link_download_encontrado:
                                 motivo_falha_final += " Work não possui outputs associados listados para tentativa."
                    else: # Se work_id é None
                        # Combina as mensagens de erro se ambas as buscas falharam
                        if erro_busca_output and erro_busca_work:
                             motivo_falha_final = f"Output: {erro_busca_output}. Work: {erro_busca_work}."
                        elif erro_busca_work: # Se só a busca por work falhou (output já tinha sido tratada)
                             motivo_falha_final = f"Work não encontrado ou erro na busca: {erro_busca_work}"


            resultados_finais.append({
                'doi': doi, # Salva o DOI original da planilha
                'core_id_output_tentado': core_id_output_tentado,
                'core_id_work_tentado': core_id_work_tentado,
                'core_id_output_do_work_tentado': core_id_output_do_work_tentado if tipo_entidade_link == "outputs (de work)" else None,
                'id_usado_para_link': id_usado_para_link,
                'tipo_entidade_link': tipo_entidade_link,
                'status_link': status_final,
                'motivo_falha': motivo_falha_final if status_final == "Falha" else "",
                'link_download': link_download_encontrado
            })
            
            # Aumenta o delay base entre o processamento de cada DOI
            # 60 segundos / 10 reqs por minuto = 6 segundos por requisição.
            # Como podemos fazer mais de uma requisição por DOI, um valor maior é mais seguro.
            print("Aguardando 7 segundos antes do próximo DOI...")
            time.sleep(7) 

        if resultados_finais:
            resultados_df = pd.DataFrame(resultados_finais)
            colunas_esperadas = ['doi', 'core_id_output_tentado', 'core_id_work_tentado', 
                                 'core_id_output_do_work_tentado', 'id_usado_para_link',
                                 'tipo_entidade_link', 'status_link', 'motivo_falha', 'link_download']
            for col in colunas_esperadas:
                if col not in resultados_df.columns:
                    resultados_df[col] = None
            
            resultados_df = resultados_df[colunas_esperadas]
            resultados_df.to_csv(ARQUIVO_SAIDA_RESULTADOS, index=False, encoding='utf-8')
            print(f"\nResultados do processamento foram salvos em: {ARQUIVO_SAIDA_RESULTADOS}")
        else:
            print("\nNenhum DOI processado para salvar.")

    except FileNotFoundError:
        pass 
    except Exception as e:
        print(f"Ocorreu um erro geral e fatal no script: {e}")
        import traceback
        traceback.print_exc()



Aguardando 7 segundos antes do próximo DOI...

--- Processando DOI 344/503: 10.1016/j.rcsar.2016.04.001 (Linha 345 da planilha) ---
Buscando outputs com DOI: 10.1016/j.rcsar.2016.04.001 (Tentativa 1/5)
Outputs ID encontrado: 82483691 para DOI: 10.1016/j.rcsar.2016.04.001
Aguardando 7 segundos antes do próximo DOI...

--- Processando DOI 345/503: 10.1590/1980-5373-MR-2017-0624 (Linha 346 da planilha) ---
Buscando outputs com DOI: 10.1590/1980-5373-MR-2017-0624 (Tentativa 1/5)
Erro 429 (Too Many Requests) ao buscar outputs para DOI 10.1590/1980-5373-MR-2017-0624.
Sem cabeçalho Retry-After. Usando backoff de 60 segundos.
Tentando novamente em 60 segundos...
Buscando outputs com DOI: 10.1590/1980-5373-MR-2017-0624 (Tentativa 2/5)
Nenhum resultado encontrado para outputs com DOI 10.1590/1980-5373-MR-2017-0624.
Fallback para 'Works' para DOI: 10.1590/1980-5373-MR-2017-0624
Buscando works com DOI: 10.1590/1980-5373-MR-2017-0624 (Tentativa 1/5)
Nenhum resultado encontrado para works com DOI 10

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d35fdc8b-8543-45dc-ae25-3ba609dd01b9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>